<h1 style="text-align:center;">Setting the active Manager NIC</h1>

<div class="alert alert-block alert-info" style="border-radius: 20px;">
<i>Hello User.!<br></i> Welcome to <b>python-iLOrest-library</b> <br> This is an example of <b>setting the active Manager NIC </b> <br> As you are here let's get it done easily.<br>
    To perform this u need to have <a href = "https://github.com/HewlettPackard/python-ilorest-library" style = "background-color:white">python-ilorest-library</a> installed in your machine.<br>
    And please keep your username and password handy!
    <br>
    <b>Also do execute each successive cell once to avoid any errors!</b>
    
</div>

<div class="alert alert-block alert-success"  style="border-radius: 20px;">
    <h3> 1. Below we are importing all the necessary python libraries</h3>
    <ul>
        <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys</a></li>
        <li>json</li>
        <li>redfish</li> 
    </ul>
</div>

In [ ]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

from get_resource_directory import get_resource_directory
#This is to take passwords as input securely
import getpass

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 2. This is about set_active_ilo_nic function</h3>
    

1. This function is used to enable/disable Ethernet interfaces on an iLO (Integrated Lights Out) management interface.<br>
<br>
    
2. The function first gets a list of available Ethernet interfaces using the Redfish API, which is a standard RESTful interface for managing servers.<br>
    <br>

3. The function first gets a list of available Ethernet interfaces using the Redfish API, which is a standard RESTful interface for managing servers.<br>
    <br>

4. If the PATCH request is successful (HTTP status 200), the function prints a success message and the response body, which contains information about the updated Ethernet interface.<br>
    <br>
5. Finally, the function reminds the user that they need to reset iLO for the change to take effect.<br>
    



 </div>

In [1]:
def set_active_ilo_nic(_redfishobj):

    ethernet_data = {}

    resource_instances = get_resource_directory(_redfishobj)
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        managers_uri = _redfishobj.root.obj['Managers']['@odata.id']
        managers_response = _redfishobj.get(managers_uri)
        managers_members_uri = next(iter(managers_response.obj['Members']))['@odata.id']
        managers_members_response = _redfishobj.get(managers_members_uri)
        manager_ethernet_interfaces = managers_members_response.obj['EthernetInterfaces']\
                                                                                    ['@odata.id']
        manager_ethernet_interfaces_response = _redfishobj.get(manager_ethernet_interfaces)
        manager_ethernet_interfaces_members = manager_ethernet_interfaces_response.\
                                                            obj['Members']
        for _member in manager_ethernet_interfaces_members:
            _tmp = _redfishobj.get(_member['@odata.id']).obj
            ethernet_data[_member['@odata.id']] = _tmp
    else:
        #Use Resource directory to find the relevant URI
        for instance in resource_instances:
            if '#EthernetInterfaceCollection.' in instance['@odata.type'] and 'Managers' in \
                                                                        instance['@odata.id']:
                ethernet_uri = instance['@odata.id']
                ethernet_interfaces = _redfishobj.get(ethernet_uri).obj['Members']
                for _ethernet_interface in ethernet_interfaces:
                    ethernet_data[_ethernet_interface['@odata.id']] = _redfishobj.\
                                                        get(_ethernet_interface['@odata.id']).dict
                break

    if ethernet_data:
        print("\n\nShowing all available ethernet management interfaces before changes:\n\n")
        for interface in ethernet_data:
            sys.stdout.write("Ethernet Management Inteface \'%s\'\n" % ethernet_data\
                                                                            [interface].get('Id'))
            sys.stdout.write("\t\'Interface Enabled\': %s\n" % ethernet_data[interface].\
                                                                        get('InterfaceEnabled'))

    for ethernet in ethernet_data:
        sys.stdout.write("Ethernet Interface: %s\n" % ethernet)
        ans = input("Would you like to enable/disable this interface? (enable/disable)\n")
        if "en" in ans:
            body = {"InterfaceEnabled": True}
        else:
            body = {"InterfaceEnabled": False}
        resp = _redfishobj.patch(ethernet, body)
        if resp.status == 400:
            try:
                print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, \
                                                                                sort_keys=True))
            except Exception as excp:
                sys.stderr.write("A response error occurred, unable to access iLO Extended "\
                                 "Message Info...")
        elif resp.status != 200:
            sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
        else:
            print("Success! You will need to reset iLO for this change to take effect.\n")
            print(json.dumps(resp.dict, indent=4, sort_keys=True))
        break


<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 3. This is Main function overview.</h3>
<b>The Main function: </b> PLEASE ENTER YOU LOGIN CREDENTIALS HERE TO PROCEED FURTHER<br>
    <ul>
        <li>Once your credentials that are enter everything a redfish obejct is created with the help of RedfishClient class and a session is created and now you will be logged in to the server.</li>
        <li>If server is down or Unreachable we get server not reachable message!</li>
        <li>If not then we make a call to <b>mount_virtual_media_iso</b> to obtain virtual media related info of system.</li>
        <li>After getting required information <i><b>Redfish.logout()</b></i> closses the particular session and logs us out of the system.</li>
    </ul>
</div>

<div class="alert alert-block alert-warning" style="border-radius: 20px;">
<b> When running on the server locally,</b>use the following values:<br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br>
    <b> When running remotely </b> connect using the secured (https://) address, account name and password to send https requests<br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

<div class="alert alert-block alert-warning" style="border-radius: 20px;">
    <h5>Instructions</h5>
    <ul>
        <li>When prompted for <i>ACCOUNT_LOGIN_NAME</i>, enter redfish standard username here  as iLO GUI actually considers this to be 'UserName'</li>
        <li>When prompted for <i>ACCOUNT_USER_NAME</i>, enter redfish login here iLO GUI actually considers this to be 'LoginName'</li>
        <li>Role_ID: It is a predefined role for the user which can take "Administrator" or "ReadOnly" or "Operator" as input. For more informaion on roles and privilages refere the table in the next cell.</li>
        <li>For more clarification on the above points please visit <i><a href="https://hewlettpackard.github.io/ilo-rest-api-docs/ilo6/#managing-ilo-6-users">iLO Users Documentation</a></i></li>
    </ul>
</div>

In [2]:
if __name__ == "__main__":
    
        SYSTEM_URL = input("Enter System URL: ")
    LOGIN_ACCOUNT = input("Enter Login account: ")
    LOGIN_PASSWORD = getpass.getpass("Enter your password: ")
    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    set_active_ilo_nic(REDFISHOBJ)
    REDFISHOBJ.logout()

NameError: name 'ServerDownOrUnreachableError' is not defined

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library: <a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.HPE ilorest-api explorer: <a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.REST API: <a href = "https://restfulapi.net/">LINK</a><br>
    4.Changing Boot Setting Documentation: <a href = "https://hewlettpackard.github.io/ilo-rest-api-docs/ilo6/?python#changing-boot-settings">LINK</a><br>
    
</div>